In [ ]:
import os
from xml.dom.pulldom import default_bufsize
from dotenv import load_dotenv
import duckdb
from collections import defaultdict
import pandas as pd
import re

In [ ]:
load_dotenv()
# conn = duckdb.connect(':memory:')
conn = duckdb.connect("citi_bike.duckdb")
conn.execute("PRAGMA threads=8;")
conn.execute("PRAGMA enable_print_progress_bar;")
conn.execute("SET preserve_insertion_order = false;")
conn.execute('INSTALL httpfs;')
conn.execute('LOAD httpfs;')
# authenication
conn.execute(f"SET s3_region='us-east-1';")
conn.execute(f"SET s3_access_key_id='{os.environ['AWS_ACCESS_KEY_ID']}';")
conn.execute(f"SET s3_secret_access_key='{os.environ['AWS_SECRET_ACCESS_KEY']}';")

In [ ]:
nyc_year_range = range(2013,2026)
jc_year_range = range(2015,2026)

In [ ]:
nyc_schema_cols_1 = [
    "trip_duration",
    # "trip_date",
    "start_time",
    "stop_time",
    "start_station_id",
    "start_station_name",
    "start_station_lat",
    "start_station_long",
    "end_station_id",
    "end_station_name",
    "end_station_lat",
    "end_station_long",
    "user_type"
]
nyc_schema_cols_2 = [
    'start_time',
    'stop_time', 
    'start_station_name', 
    'start_station_id', 
    'end_station_name', 
    'end_station_id', 
    'start_station_lat', 
    'start_station_long', 
    'end_station_lat', 
    'end_station_long',
    'user_type'
]
jc_schema_cols_1 = [
    'trip_duration',
    'start_time',
    'stop_time',
    'start_station_id',
    'start_station_name',
    'start_station_lat',
    'start_station_long',
    'end_station_id',
    'end_station_name',
    'end_station_lat',
    'end_station_long',
    'user_type'
]
jc_schema_cols_2 = [
    'start_time',
    'stop_time',
    'start_station_name',
    'start_station_id',
    'end_station_name',
    'end_station_id',
    'start_station_lat',
    'start_station_long',
    'end_station_lat',
    'end_station_long',
    'user_type',
]

schema_cols_map = {"nyc": {"2013":nyc_schema_cols_1, "2014":nyc_schema_cols_1, "2015":nyc_schema_cols_1, "2016":nyc_schema_cols_1, "2017":nyc_schema_cols_1, "2018":nyc_schema_cols_1, "2019":nyc_schema_cols_1, "2020":nyc_schema_cols_2, "2021":nyc_schema_cols_2, "2022":nyc_schema_cols_2, "2023":nyc_schema_cols_2, "2024":nyc_schema_cols_2, "2025":nyc_schema_cols_2},
            "jc": {"2015": jc_schema_cols_1, "2016": jc_schema_cols_1, "2017": jc_schema_cols_1, "2018": jc_schema_cols_1, "2019": jc_schema_cols_1, "2020": jc_schema_cols_1, "2021": jc_schema_cols_2, "2022": jc_schema_cols_2, "2023": jc_schema_cols_2, "2024": jc_schema_cols_2, "2025": jc_schema_cols_2}}

In [ ]:
col_mapping = {"nyc": {"2013" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'], 
                                     "filter_clause" : 'WHERE starttime IS NOT NULL AND stoptime IS NOT NULL AND "start station id" IS NOT NULL AND "start station latitude" IS NOT NULL AND "start station longitude" IS NOT NULL AND "end station id" IS NOT NULL AND "end station latitude" IS NOT NULL AND "end station longitude" IS NOT NULL'},
                           "2014" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'],
                                    "filter_clause" : ""},
                           "2015" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'], 
                                     "filter_clause" : ""},
                           "2016" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'], 
                                     "filter_clause" : ""},
                           "2017" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'], 
                                    "filter_clause" : 'WHERE "start station latitude" IS NOT NULL AND "end station latitude" IS NOT NULL AND "end station longitude" IS NOT NULL'},
                           "2018" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'],
                                    "filter_clause" : 'WHERE "start station id" IS NOT NULL AND "end station id" IS NOT NULL'},
                           "2019" : {"col_names" : ["tripduration as trip_duration","starttime as start_time","stoptime as stop_time", '"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','usertype as user_type'], 
                                    "filter_clause" : 'WHERE "start station id" IS NOT NULL AND "end station id" IS NOT NULL'},
                           "2020" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_station_id IS NOT NULL AND end_lat IS NOT NULL"},
                           "2021" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_station_id IS NOT NULL AND end_lat IS NOT NULL"},
                           "2022" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_station_id IS NOT NULL AND end_lat IS NOT NULL"},
                           "2023" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_lat IS NOT NULL AND end_lat IS NOT NULL"},
                           "2024" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_lat IS NOT NULL AND end_lat IS NOT NULL"},
                           "2025" : {"col_names" : ["started_at as start_time","ended_at as stop_time","start_station_name","start_station_id","end_station_name","end_station_id","start_lat as start_station_lat","start_lng as start_station_long","end_lat as end_station_lat","end_lng as end_station_long","member_casual as user_type"],
                                     "filter_clause" : "WHERE start_lat IS NOT NULL AND end_lat IS NOT NULL"}
                           }, 
                    "jc": {"2015" : {"col_names" : ['"trip duration" as trip_duration','"start time" as start_time','"stop time" as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"user type" as user_type'],
                                     "filter_clause" : ""},
                           "2016" : {"col_names" : ['"trip duration" as trip_duration','"start time" as start_time','"stop time" as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"user type" as user_type'],
                                     "filter_clause" : ""},
                           "2017" : {"col_names" : ['"tripduration" as trip_duration','starttime as start_time','stoptime as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"usertype" as user_type'],
                                     "filter_clause" : ""},
                           "2018" : {"col_names" : ['"tripduration" as trip_duration','starttime as start_time','stoptime as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"usertype" as user_type'],
                                     "filter_clause" : ""},
                           "2019" : {"col_names" : ['"tripduration" as trip_duration','starttime as start_time','stoptime as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"usertype" as user_type'],
                                     "filter_clause" : ""},
                           "2020" : {"col_names" : ['"tripduration" as trip_duration','starttime as start_time','stoptime as stop_time','"start station id" as start_station_id','"start station name" as start_station_name','"start station latitude" as start_station_lat','"start station longitude" as start_station_long','"end station id" as end_station_id','"end station name" as end_station_name','"end station latitude" as end_station_lat','"end station longitude" as end_station_long','"usertype" as user_type'],
                                     "filter_clause" : ""},
                           "2021" : {"col_names" : ['started_at as start_time', 'ended_at as stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat as start_station_lat', 'start_lng as start_station_long', 'end_lat as end_station_lat', '"end_lng" as end_station_long', 'member_casual as user_type'],
                                     "filter_clause" : "WHERE start_lat IS NOT NULL AND end_lat IS NOT NULL"},
                           "2022" : {"col_names" : ['started_at as start_time', 'ended_at as stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat as start_station_lat', 'start_lng as start_station_long', 'end_lat as end_station_lat', '"end_lng" as end_station_long', 'member_casual as user_type'],
                                     "filter_clause" : "WHERE end_lat IS NOT NULL"},
                           "2023" : {"col_names" : ['started_at as start_time', 'ended_at as stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat as start_station_lat', 'start_lng as start_station_long', 'end_lat as end_station_lat', '"end_lng" as end_station_long', 'member_casual as user_type'],
                                     "filter_clause" : "WHERE end_lat IS NOT NULL"},
                           "2024" : {"col_names" : ['started_at as start_time', 'ended_at as stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat as start_station_lat', 'start_lng as start_station_long', 'end_lat as end_station_lat', '"end_lng" as end_station_long', 'member_casual as user_type'],
                                     "filter_clause" : "WHERE end_lat IS NOT NULL"},
                           "2025" : {"col_names" : ['started_at as start_time', 'ended_at as stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat as start_station_lat', 'start_lng as start_station_long', 'end_lat as end_station_lat', '"end_lng" as end_station_long', 'member_casual as user_type'],
                                     "filter_clause" : "WHERE start_lat IS NOT NULL AND end_lat IS NOT NULL"}}}


In [ ]:
# annual total rides -> source -> city -> year -> num
# monthly -> cleaned dataset -> fuk it
total_rides = {"nyc": {},"jc":{}}
for year in nyc_year_range:
    total_rides["nyc"][str(year)] = 0
for year in jc_year_range:
    total_rides["jc"][str(year)] = 0

In [ ]:
nyc_views = {} # year [str] -> view_name [str]

In [ ]:
def prune_local_tables_in_disk(city=None):
    # fetch all table names once
    all_tables = [t[0] for t in conn.execute("SHOW TABLES;").fetchall()]
    if city == "nyc":
        all_tables = list(filter(lambda x: x.startswith("nyc"),all_tables))
    elif city == "jc":
        all_tables = list(filter(lambda x: x.startswith("jc"),all_tables))
    else:
        all_tables = list(filter(lambda x: x.startswith("nyc") or x.startswith("jc"),all_tables))
    for table in all_tables:
        print(f"Dropping table {table}")
        conn.execute(f"DROP TABLE IF EXISTS {table}")
def show_tables_on_disk():
    print([t[0] for t in conn.execute("SHOW TABLES;").fetchall()])

In [ ]:
for city in ["nyc","jc"]:
    year_range = nyc_year_range if city == "nyc" else jc_year_range
    for year in year_range:
        table_name = table_name = f"{city}_{year}"
        tables = [c[0] for c in conn.execute("SHOW TABLES").fetchall()]
        if table_name not in tables:
            print(f"⏳ Creating table {table_name} from S3 ...")
            conn.execute(f"""
                    CREATE OR REPLACE VIEW raw_view AS SELECT * FROM {f"read_parquet('s3://citibike-nycdata/parquet_files/{city}_files/{year}/*')"}
                        """)
            # ['column_name', 'column_type', 'min', 'max', 'approx_unique', 'avg', 'std', 'q25', 'q50', 'q75', 'count', 'null_percentage']
            result = conn.execute("SELECT COUNT(*) FROM raw_view").fetchall()
            raw_view_total = result[0][0]
            total_rides[city][str(year)] = raw_view_total
            print(f"Total entries found for {year} -> {raw_view_total}")
            print("raw_view columns -> \n",[(col[0],col[1]) for col in conn.execute("DESCRIBE raw_view").fetchall()])
            select_cols = ','.join(col_mapping[city][str(year)]["col_names"])
            result = conn.execute(f'CREATE OR REPLACE VIEW temp_view AS SELECT {select_cols} FROM raw_view {col_mapping[city][str(year)]["filter_clause"]}')
            temp_view_cols = [(col[0],col[1]) for col in conn.execute("DESCRIBE temp_view").fetchall()]
            print("temp_view columns -> \n",temp_view_cols)
            # check against schema
            print("checking schema validity")
            schema_cols = schema_cols_map[city][str(year)]
            assert len(temp_view_cols) == len(schema_cols), "Incorrect column count in temp view!"
            for col_name in schema_cols:
                assert col_name in [c for c,_ in temp_view_cols], f"col {col_name} missing in temp view"
            print("done checking schema validity :))")
            # end check against schema
            # print("sample from temp_view ->")
            # print(conn.execute("select * from temp_view limit 5").fetchall())
            # print([(e[0],e[-1]) for e in conn.execute("SUMMARIZE(SELECT * FROM temp_view);").fetchall()])
            # begin nullity check
            
            query = []
            for c in schema_cols:
                query.append(f"COUNT(*) - COUNT({c})")
            result = conn.execute(f"""
                            SELECT {','.join(query)} FROM temp_view
                                """).fetchall()
            for i,col in enumerate(schema_cols):
                print(f"Null Count -- {col} -> ",result[0][i])
                assert result[0][i] == 0, "nil value unhandled!!"
            
            # end nullity check
            temp_view_total = conn.execute("SELECT COUNT(*) FROM temp_view").fetchall()[0][0]
            print(f"reduced {raw_view_total} by {raw_view_total-temp_view_total} -> {temp_view_total}")
            print("sample query -> ")
            print(conn.execute("SELECT * FROM temp_view LIMIT 5").fetchall())
            print("attempting to add view")
            conn.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM temp_view;")
            nyc_views[str(year)] = table_name
            print("done adding view...")
            # filter to new view entries where any of the following col c1,c2,c3,c4 are null
            # print([c[0] for c in conn.description])
            # print(f"{'Column':25} | {'Type':10} | {'Null %':>7} | {'Count':>8} | {'Avg':>10} | {'Std':>10}")
            # print("-" * 80)
            # for e in result:
            #     col_name, col_type, _, _, _, avg, std, _, _, _, count, null_pct = e
            #     print(f"{col_name:25} | {col_type:10} | {float(null_pct):7.2f} | {count:8} | {avg or '':>10} | {std or '':>10}")
            print(f"✅ {table_name} created & persisted.")
        else:
            print(f"⚡ {table_name} already exists — skipping.")
        print("--------------------------------------------------------------------")


In [ ]:
# booopppppppp

In [ ]:
# everything above this cell is expensive af!!!!

In [ ]:
nyc_annual_total_rides = []
for year in nyc_year_range:
    res = conn.execute(f"SELECT COUNT(*) FROM nyc_{year}").fetchall()[0][0]
    nyc_annual_total_rides.append({"year": year, "total_rides": res})
df = pd.DataFrame(nyc_annual_total_rides)
df.to_csv("citibike_rides_per_year.csv", index=False)

In [ ]:
for year in nyc_year_range:
    res = conn.execute(f"DESCRIBE nyc_{year}").fetchall()
    print(res)